In [1]:
import time
import math

import numpy as np
import pandas as pd
from biotite.database import entrez
from biotite.database import uniprot
import biotite.sequence.io.fasta as fasta

In [2]:
# Unfortunately, to my knowledge, downloading all VACV interactions from
# the HVIDB database at once is not possible
# Hence, they have been downloaded in packages of up to 100 interactions
# It is investigated whether the 5 packages are all different from each
# other so as to rule out errors during the manual download, such as
# accidentally downloading the same package twice
# As a first step, the individual packages are loaded into the Jupyter
# notebook
HVIDB_package_1 = pd.read_csv("HVIDB_VACV_interactions_1.csv")
HVIDB_package_2 = pd.read_csv("HVIDB_VACV_interactions_2.csv")
HVIDB_package_3 = pd.read_csv("HVIDB_VACV_interactions_3.csv")
HVIDB_package_4 = pd.read_csv("HVIDB_VACV_interactions_4.csv")
HVIDB_package_5 = pd.read_csv("HVIDB_VACV_interactions_5.csv")

# According to the HVIDB database, the total amount of interactions is
# 456
# It is checked whether this also applies to the five downloaded
# packages
packages_list = [
    HVIDB_package_1,
    HVIDB_package_2,
    HVIDB_package_3,
    HVIDB_package_4,
    HVIDB_package_5
]

assert sum(map(len, packages_list)) == 456, (
    "An error has been made during the manual download of the "
    "interaction packages!"
)

In [3]:
# Now, verify that all downloaded interaction packages are distinct from
# one another
# Unfortunately, checking equality for more than two DataFrames at once
# is not possible to the best of my knowledge
# Hence, it is resorted to for-loops
package_equality = False

for i, package in enumerate(packages_list):
    # For obvious reasons, equality checks are only performed between
    # different interaction packages
    # Moreover, redundant equality checks are avoided
    for j in range(i + 1, 5):
        package_equality = package_equality or package.equals(packages_list[j])

assert package_equality == False, (
    "Identical interaction packages have erroneously been downloaded!"
)

In [4]:
# Now, for the sake of convenience, concatenate the five individual
# interaction packages to generate one coherent CSV file
HVIDB_VACV_interactions_df = pd.concat(packages_list, ignore_index=True)

assert (
    len(HVIDB_VACV_interactions_df) == 456
    and
    len(HVIDB_VACV_interactions_df.columns) == 4
), "A mistake was done during DataFrame concatenation!"

In [9]:
# Save the coherent DataFrame as a CSV file
HVIDB_VACV_interactions_df.to_csv(
    "all_HVIDB_VACV_interactions.csv", index=False
)

In [7]:
# Determine the unique values for the column "PubMed ID", i.e. determine
# the publications interaction information was retrieved from
pre_unique_pubs = np.unique(HVIDB_VACV_interactions_df["PubMed ID"])
# Some cells contain multiple PubMed IDs separated by commata, which
# requires special handling
pre_unique_single_pubs = []
for pub_id in pre_unique_pubs:
    if "," in pub_id:
        pre_unique_single_pubs += pub_id.split(",")
    else:
        pre_unique_single_pubs.append(pub_id)
unique_pubs = np.unique(pre_unique_single_pubs)
print(len(unique_pubs))
print(unique_pubs)

39
['11129635' '11238845' '11878931' '12566418' '12960323' '14963148'
 '15215253' '15518812' '15767367' '15998638' '16003387' '1610791'
 '16254338' '16439990' '16840345' '17074758' '17123957' '17166913'
 '17485524' '17681535' '18005740' '18208323' '18266467' '18353424'
 '18551131' '18604270' '18636090' '18971339' '19637933' '19913487'
 '21915095' '21931555' '22810585' '24550280' '7609027' '9199350' '9431994'
 '9792841' '9824158']


In [5]:
# Determine the amount of genes targeted by siRNAs/esiRNAs in the VACV
# screen
# Low-molecular weight organic compounds are not taken into account as
# despite having the same final outcome as siRNAs, i.e. the inhibition
# of the action of proteins, low-molecular weight compounds and siRNAs
# differ from one another in their mode of action: The former come into
# play after protein translation has taken place, whereas the latter
# unfold their effect on the post-transcriptional level by targeting
# mRNAs
# Hence, exactly the same effect is not expected
single_pooled_siRNA_and_esiRNA_STRING_IDs_df = pd.read_csv(
    "single_pooled_siRNA_and_esiRNA_STRING_IDs.csv",
    delimiter="\t"
)

n_targets = len(single_pooled_siRNA_and_esiRNA_STRING_IDs_df)

print(
    "Amount of human proteins targeted in the VACV screen by "
    f"single/pooled siRNAs as well as esiRNAs: {n_targets:,}"
)

Amount of human proteins targeted in the VACV screen by single/pooled siRNAs as well as esiRNAs: 18,395


In [6]:
# Now, determine the amount of human genes interrogated in the VACV
# screen that engage in PPIs recorded by HVIDB
# To this end, the protein target names have to be converted in
# UniProtIDs as they are employed by HVIDB
# This is achieved via the ID mapping GUI on
# https://www.uniprot.org/id-mapping
VACV_screen_human_targets_series = (
    single_pooled_siRNA_and_esiRNA_STRING_IDs_df["queryItem"]
)

VACV_screen_human_targets_series.to_csv(
    "VACV_screen_human_targets_single_pooled_siRNA_and_esiRNA_gene_names.txt",
    header=False,
    index=False
)

In [1]:
# Using the parameters "from: UniProt/Gene Name",
# "to: UniProtKB-Swiss-Prot" and "taxId: 9606", 18,185 of the 18,396
# human protein IDs were mapped to 18,645 results, whereas 213 ID were
# not mapped
# This is subjected to close scrutiny
#
# For the ALG1L gene, the official gene symbol has apparently been
# changed after the VACV screen has been conducted; its new official
# gene symbol is ALG1L1P while its NCBI Gene ID remained unchanged
# (200810)
#
# Regarding the gene AKAP2, the NCBI Gene ID has been changed from 11217
# to 445815; note that AKAP2 is only an alias and that the offical gene
# symbol is PALM2AKAP2
#
# With respect to the gene PPAN-P2RY11, neither the official gene symbol
# nor the NCBI Gene ID have been altered after the VACV screen, but the
# respective UniProtKB entry belongs to the UniProtKB/TrEMBL section,
# i.e. has not been manually reviewed. Consequently, using the setting
# "to: UniProtKB-Swiss-Prot" does not yield any UniProtID for this gene
# It becomes apparent that the most convenient way of obtaining
# UniProtIDs in one single step is to use the settings
# "From database: Genomic annotation databases/GeneID" (Gene ID is the
# same as NCBI Gene ID) and "To database: UniProt/UniProtKB" (choosing
# UniProtKB includes both UniProtKB sections, i.e. UniProtKB/Swiss-Prot
# and UniProtKB/TrEMBL)
#
# With regard to the gene CDY, the specified NCBI Gene ID is still valid
# (203611), but its official gene symbol is CDY2B
#
# As to the gene FLJ40296, the specified NCBI Gene ID is still valid
# (122183), but its official gene symbol is PRR20A
#
# Regarding the gene MGC39606, the specified NCBI Gene ID is still valid
# (399668), but its official gene symbol is SMIM10L2A
#
# etc.

In [ ]:
# Instead of doing this investigation manually, it is accomplished in an
# automated fashion by means of the Biotite bioinformatics library
# Biotite provides functionalities allowing to access various NCBI
# databases in a programmatic manner, i.e. from within Python scripts

# Two columns of interest for the following endeavour are "ID" as well
# as "ID_manufacturer"
# Upon closer scrutiny, it became apparent that contrary to their names,
# both features basically harbour the same information, namely the NCBI
# Gene ID of the targeted gene
# However, they differ in that while the feature "ID" is populated
# depending on whether the respective experiment was successful or not,
# "ID_manufacturer" is continuously populated, irrespective of the
# experiment's outcome
# Hence, the entries of the "ID_manufacturer" feature are employed in
# order to query NCBI's gene database
# As a first step, it is checked whether all entries of the
# "ID_manufacturer" feature are indeed populated with values or whether
# there are isolated occurrences of "Not available"
# Note that the features "ID" and "ID_manufacturer" only harbour NCBI
# Gene IDs in the case of siRNA, pooled siRNA and esiRNA

# Load the screen data
# Bear in mind that for certain columns, the data type has to be
# manually specified
dtype_dict = {
    "Ensembl_ID_OnTarget_Ensembl_GRCh38_release_87": str,
    "Ensembl_ID_OnTarget_NCBI_HeLa_phs000643_v3_p1_c1_HMB": str,
    "Gene_Description": str,
    "ID": str,
    "ID_OnTarget_Ensembl_GRCh38_release_87": str,
    "ID_OnTarget_Merge": str,
    "ID_OnTarget_NCBI_HeLa_phs000643_v3_p1_c1_HMB": str,
    "ID_OnTarget_RefSeq_20170215": str,
    "ID_manufacturer": str,
    "Name_alternatives": str,
    "PLATE_QUALITY_DESCRIPTION": str,
    "RefSeq_ID_OnTarget_RefSeq_20170215": str,
    "Seed_sequence_common": str,
    "WELL_QUALITY_DESCRIPTION": str,
    "siRNA_error": str,
    "siRNA_number": str,
    "Precursor_Name": str
}

# Dask DataFrames exhibit a peculiarity regarding the index labels: By
# default, the index labels are integers, just as with Pandas
# DataFrames. However, unlike Pandas DataFrames, the index labels do not
# monotonically increase from 0, but restart at 0 for each partition,
# thereby resulting in duplicated index labels (Dask subdivides a Dask
# DataFram into multiple so-called partitions as the whole idea behind
# Dask is to handle large data sets in a memory-efficient way, https://
# docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.reset_
# index.html)
# Hence, performing operations with Dask DataFrames might potentially
# raise the `ValueError: cannot reindex on an axis with duplicate
# labels` error
# In this case, loading the entire data set into a Pandas DataFrame is
# feasible, which is why this is preferred to loading it into a Dask
# DataFrame (strangely enough, this has not been possible in the very
# beginning, which is why Dask was used in the first place)
main_csv_df = pd.read_csv(
    "VacciniaReport_20170223-0958_ZScored_conc_and_NaN_adjusted.csv",
    sep="\t",
    dtype=dtype_dict
)

# Bear in mind that due to operator precedence, i.e. "|" (logical OR)
# having precedence over equality checks, the equality checks have to be
# surrounded by parentheses
single_pooled_siRNA_and_esiRNA_df = main_csv_df.loc[
    (main_csv_df["WellType"] == "SIRNA")
    |
    (main_csv_df["WellType"] == "POOLED_SIRNA")
    |
    (main_csv_df["WellType"] == "ESIRNA")
]

In [8]:
single_pooled_siRNA_and_esiRNA_IDs = single_pooled_siRNA_and_esiRNA_df[
    "ID_manufacturer"
]

# Bear in mind that the in operator cannot be used in the context of
# Pandas Serieses as it refers to the index instead of the data itself
# Thus, the `in` operator is employed in conjunction with the `values`
# attribute of Pandas Serieses, which returns the Series as a NumPy
# array
assert "Not available" not in single_pooled_siRNA_and_esiRNA_IDs.values, (
    "Contrary to our expectations, the entry \"Not available\" does "
    "occur in the feature \"ID_manufacturer\"!"
)

AssertionError: Contrary to our expectations, the entry "Not available" does occur in the feature "ID_manufacturer"!

In [8]:
ID_change_str = "This record was replaced with GeneID:"

# Iterate over the DataFrame rows using the iterrows() method
for i, row in main_csv_df.iterrows():
    # Incorporating kind of a numeric progress bar
    if i % 10000 == 0:
        print(i)
    well_type = row["WellType"]
    if (
        well_type == "SIRNA"
        or
        well_type == "POOLED_SIRNA"
        or
        well_type == "ESIRNA"
    ):
        gene_name = row["Name"]
        NCBI_Gene_ID = row["ID_manufacturer"]
        
        # Query NCBI's gene database with the Gene ID currently dealt
        # with
        # Code execution is suspended for two seconds in order to avoid
        # server-side errors
        # The VACV data set comprises 132,066 measurements involving
        # single siRNAs, pooled siRNAs and esiRNAs
        # For each of those measurements, the NCBI database is queried
        # at least once, entailing the suspension of code execution for
        # two seconds; in total, this amounts to a "waiting time" of
        # 264,132 seconds, which corresponds to slightly more than three
        # days; this period of time is acceptable
        time.sleep(2)
        NCBI_entry = entrez.fetch_single_file(
            uids=[NCBI_Gene_ID],
            file_name=None,
            db_name="gene",
            ret_type="",
            ret_mode="text"
        )

        # As the file_name is specified to be None, Biotite's
        # fetch_single_file() function returns a StringIO object
        # It's content can be accessed via the getvalue() method
        # Note that the getvalue() method is preferred to the read()
        # method as the latter moves the cursor to the last index so
        # that repeatedly using the read() method returns an empty
        # string
        NCBI_entry_str = NCBI_entry.getvalue()

        # Different approaches are necessary depending on whether merely
        # the official gene symbol was altered, whereas the NCBI Gene ID
        # remained unchanged, or the record has been replaced altogether
        # with an entirely new ID
        if ID_change_str in NCBI_entry_str:
            # The respective record has been replaced altogether with a
            # new ID
            # Hence, the new ID is retrieved and used to query NCBI's
            # gene database
            NCBI_entry_str_list = NCBI_entry_str.split("\n")
            # For some strange reason, the string retrieved from the
            # NCBI entry contains blank lines; they are removed
            while "" in NCBI_entry_str_list:
                NCBI_entry_str_list.remove("")
            
            # The new ID is comprised in the penultimate list element
            # and conveniently enough separated from the preceding
            # string by a space character
            new_gene_ID = NCBI_entry_str_list[-1].split()[-1]
            main_csv_df.at[i, "ID"] = new_gene_ID
            main_csv_df.at[i, "ID_manufacturer"] = new_gene_ID

            # Again, in a bid to prevent the occurrence of server-side
            # errors, code execution is suspended for two seconds
            time.sleep(2)
            NCBI_entry = entrez.fetch_single_file(
                uids=[new_gene_ID],
                file_name=None,
                db_name="gene",
                ret_type="",
                ret_mode="text"
            )
            NCBI_entry_str = NCBI_entry.getvalue()
            NCBI_entry_str_list = NCBI_entry_str.split("\n")
            while "" in NCBI_entry_str_list:
                NCBI_entry_str_list.remove("")
            
            # The official gene symbol is comprised in the first list
            # element, but is preceded by the string "1. ",
            # which encompasses three characters
            official_gene_symbol = NCBI_entry_str_list[0][3:]

            main_csv_df.at[i, "Name"] = official_gene_symbol
        else:
            # Remove blank lines from the string retrieved from the NCBI
            # entry
            NCBI_entry_str_list = NCBI_entry_str.split("\n")
            while "" in NCBI_entry_str_list:
                NCBI_entry_str_list.remove("")
            
            # Following the removal of empty strings, the official gene
            # symbol is represented by the first list element, but it is
            # preceded by the string "1. ", which encompasses three
            # characters
            # Hence, prior to comparing the gene names provided by the
            # VACV screen data set to the official gene symbols, the
            # first list element has to be sliced accordingly
            official_gene_symbol = NCBI_entry_str_list[0][3:]
            if gene_name != official_gene_symbol:
                main_csv_df.at[i, "Name"] = official_gene_symbol
            
            # Irrespective of whether the gene name provided by the VACV
            # data set and the official gene symbol match or not, the
            # corresponding cell of the "ID" feature is populated with
            # the NCBI Gene ID harboured by the "ID_manufacturer"
            # feature (remember that cells of the "ID" feature are not
            # continuously populated)
            main_csv_df.at[i, "ID"] = NCBI_Gene_ID

KeyboardInterrupt: 

In [ ]:
# As a last step, save the new Pandas DataFrame to a new CSV file
main_csv_df.to_csv(
    "Vaccinia_Report_NCBI_Gene_IDs_and_official_gene_symbols_updated.csv",
    index=False
)

In [2]:
# A UniProt reference proteome has been found, which encompasses 219
# proteins (the respective web page claims that it encompasses 218
# proteins, but a closer look at the FASTA file reveals that they are
# 219 instead)
# It is checked whether these 219 proteins comprise all interacting VACV
# proteins recorded by HVIDB or whether HVIDB contains VACV proteins not
# comprised in the reference proteome
# First, perform a numerical check
HVIDB_VACV_interactions_df = pd.read_csv(
    "all_HVIDB_VACV_interactions.csv"
)
print(HVIDB_VACV_interactions_df.columns)

Index(['Human-virus PPI', 'Experimental System', 'PubMed ID', 'Virus acronym'], dtype='object')


In [3]:
# The VACV UniProt IDs are comprised in the first column bearing the
# name "Human-virus PPI"; to be more precise, the first column
# represents an interaction pair between a human and a VACV protein as a
# combination of the respective UniProt IDs, which are separated by a
# hyphen
# The first UniProt ID represents a human protein, whereas the second
# UniProt ID represents a VACV protein
interaction_pair_IDs = HVIDB_VACV_interactions_df[
    "Human-virus PPI"
].to_list()

In [4]:
VACV_interaction_IDs = [
    pair.split("-")[1] for pair in interaction_pair_IDs
]

assert len(interaction_pair_IDs) == len(VACV_interaction_IDs), (
    "Something went wrong!"
)

In [5]:
# Bear in mind that one and the same protein can engage in more than one
# PPI, which also applies to VACV
# Hence, duplicate UniProt IDs are discarded
VACV_unique_interaction_IDs = np.unique(VACV_interaction_IDs).tolist()

In [6]:
# Scrutinising the FASTA file revealed that it indeed comprises 218
# sequences as claimed by the UniProt web page (the ">" character, which
# represents the start of a header, has an additional occurrence in an
# entry description)
assert len(VACV_unique_interaction_IDs) <= 218, (
    "The amount of VACV proteins involved in interspecies PPI recorded "
    "by HVIDB exceeds the size of the UniProt reference proteome "
    "(219)."
)

print(
    "Total amount of VACV proteins involved in interspecies PPI "
    f"recorded by HVIDB: {len(VACV_unique_interaction_IDs)}"
)

Total amount of VACV proteins involved in interspecies PPI recorded by HVIDB: 61


In [7]:
# Now, beyond a simple numerical check, it is investigated for each VACV
# protein in HVIDB individually whether it is included in the UniProt
# reference proteome or not
# Conveniently enough, the header contains the respective protein's
# UniProt ID, thereby obviating the necessity of ID conversion
# Gather the UniProt IDs of the proteins comprised in the UniProt
# reference proteome
ref_proteome_fasta = fasta.FastaFile.read(
    "uniprotkb_proteome_UP000000344_VACV_WR_04_11_2024.fasta"
)
ref_proteome_dict = fasta.get_sequences(ref_proteome_fasta)
ref_proteome_headers = list(ref_proteome_dict.keys())
ref_IDs = [header.split("|")[1] for header in ref_proteome_headers]

absence_list = [0] * len(VACV_unique_interaction_IDs)

for i, query_ID in enumerate(VACV_unique_interaction_IDs):
    if query_ID not in ref_IDs:
        absence_list[i] = 1

assert sum(absence_list) == 0, (
    f"{sum(absence_list)} proteins involved in interspecies PPIs "
    "recorded by HVIDB do not occur in the UniProt reference proteome!"
)

AssertionError: 18 proteins involved in interspecies PPIs recorded by HVIDB do not occur in the UniProt reference proteome!

In [8]:
# Determine the UniProt IDs of those 18 proteins not comprised in the
# reference proteome
outcast_protein_indices = np.nonzero(absence_list)
outcast_protein_IDs = np.array(VACV_unique_interaction_IDs)[
    outcast_protein_indices
]

In [9]:
print(outcast_protein_IDs)

['A4GDF4' 'O57173' 'O57263' 'P04299' 'P20505' 'P20639' 'P68318' 'P68451'
 'P68467' 'Q1M1E0' 'Q49PX0' 'Q49QD4' 'Q711A3' 'Q76RC6' 'Q76ZR2' 'Q86638'
 'Q8BDF8' 'Q8UYL3']


In [3]:
# Upon closer scrutiny, it becomes apparent that while one "outcast"
# protein ID (P04299) belongs to VACV strain Western Reserve and is not
# included in the reference genome, the remaining ones belong to either
# VACV in general or other strains, such as VACV strain Ankara or VACV
# strain Copenhagen
# As a consequence, two measures are taken, the first of which consists
# of discarding the VACV reference proteome in favour of the entirety of
# VACV proteins available on UniProt (440 in total as of 5th November
# 2024) and the second of which involves discarding the interspecies
# PPIs not specific to VACV strain Western Reserve from the HVIDB data
# set
# The aforementioned 440 VACV proteines are downloaded from UniProt in
# the FASTA format including both canonical sequences and isoforms (i.e.
# "FASTA (canonical & isoform)" has been chosen as "Format" option)
uniprot_IDs_to_exclude = [
    'A4GDF4', 'O57173', 'O57263', 'P04299', 'P20505', 'P20639',
    'P68318', 'P68451', 'P68467', 'Q1M1E0', 'Q49PX0', 'Q49QD4',
    'Q711A3', 'Q76RC6', 'Q76ZR2', 'Q86638', 'Q8BDF8', 'Q8UYL3'
]
# Determine the amount of interspecies interactions the outcast UniProt
# IDs are involved in in order to perform a sanity check later on
n_outcast_interactions = sum([
    int_pair.split("-")[1] in uniprot_IDs_to_exclude
    for int_pair in HVIDB_VACV_interactions_df["Human-virus PPI"]
])

HVIDB_VACV_WR_interactions_df = HVIDB_VACV_interactions_df[
    ~np.array([
        int_pair.split("-")[1] in uniprot_IDs_to_exclude
        for int_pair in HVIDB_VACV_interactions_df["Human-virus PPI"]
    ])
]

assert (
    len(HVIDB_VACV_WR_interactions_df) == (
        len(HVIDB_VACV_interactions_df) - n_outcast_interactions
    )
), (
    "Something went wrong while filtering out interspecies "
    "interactions involving the outcast UniProt IDs!"
)

In [4]:
# Save the DataFrame containing exclusively VACV proteins specific to
# the Western Reserve strain to a CSV file
HVIDB_VACV_WR_interactions_df.to_csv(
    "all_HVIDB_VACV_WR_interactions.csv",
    index=False
)

In [5]:
# Output the total amount of interspecies PPIs specific to VACV strain
# Western Reserve
print(
    "The total amount of interspecies PPIs specific to VACV strain "
    f"Western Reserve is {len(HVIDB_VACV_WR_interactions_df)}."
)

The total amount of interspecies PPIs specific to VACV strain Western Reserve is 415.


In [15]:
# Create a FASTA file containing all VACV WR proteins occurring in the
# HVIDB data set
# This is done by first retrieving the FASTA entries of the individual
# proteins from UniProt and subsequently adding them to an empty FASTA
# file
VACV_WR_unique_prots = np.unique(
    [
        int_pair.split("-")[1]
        for int_pair in HVIDB_VACV_WR_interactions_df["Human-virus PPI"]
    ]
).tolist()

uniprot_entries_list = uniprot.fetch(
    ids=VACV_WR_unique_prots,
    format="fasta"
)

VACV_WR_prots_fasta_file = fasta.FastaFile()

for io_object in uniprot_entries_list:
    # As a list of IDs is passed to "uniprot.fetch()" and no target path
    # is specified, a list of StringIO objects is returned which can be
    # iterated over
    current_fasta_file = fasta.FastaFile.read(io_object)
    # Each file contains only one entry; hence, the first and only
    # element is retrieved from the iterator returned by the `items()`
    # method
    header, seq_str = list(current_fasta_file.items())[0]

    # Finally, append the entry to the FASTA file
    VACV_WR_prots_fasta_file[header] = seq_str

assert len(VACV_WR_prots_fasta_file) == len(VACV_WR_unique_prots), (
    "Something went wrong while creating the FASTA file for the VACV "
    "strain Western Reserve proteins!"
)

In [18]:
# Save the FASTA file harbouring all VACV strain Western Reserve
# proteins to disk
VACV_WR_prots_fasta_file.write("VACV_WR_prots_in_HVIDB.fasta")

In [67]:
# In retrospect, it probably would have been safer to just look the
# sequences up in the FASTA file encompassing all VACV strain Western
# Reserve (VACV WR) proteins (sometimes, when querying databases,
# incomplete information transmission may occur)
# Therefore, it is investigated whether complete information
# transmission took place by comparing the sequences obtained by
# database quering to the ones deposited in the FASTA
all_VACV_WR_prots_fasta = fasta.FastaFile.read(
    "uniprotkb_taxonomy_id_10254_all_VACV_WR_prots_05_11_2024.fasta"
)
all_VACV_WR_prots_dict = fasta.get_sequences(all_VACV_WR_prots_fasta)

correct_seqs_list = [False] * len(VACV_WR_prots_fasta_file)

for i, (header, seq_str) in enumerate(VACV_WR_prots_fasta_file.items()):
    try:
        ref_seq = str(all_VACV_WR_prots_dict[header])
    except KeyError:
        print("One header was incompletely transmitted!")
        continue
    
    if seq_str == ref_seq:
        correct_seqs_list[i] = True

assert all(correct_seqs_list), (
    "Complete information transmission did not occur for all proteins!"
)

In [2]:
HVIDB_VACV_WR_interactions_df = pd.read_csv(
    "all_HVIDB_VACV_WR_interactions.csv"
)

In [3]:
# Repeat the procedure for the human counterpart, i.e. the human
# proteins involved in interspecies PPIs recorded by HVIDB
human_unique_prots_in_HVIDB = np.unique(
    [
        int_pair.split("-")[0]
        for int_pair in HVIDB_VACV_WR_interactions_df["Human-virus PPI"]
    ]
).tolist()

# As the total amount of human proteins is way too large to be processed
# in one single database query (357), the corresponding list is split
# into chunks each encompassing 20 UniProt IDs
CHUNK_SIZE = 20
n_chunks = math.ceil(len(human_unique_prots_in_HVIDB) / CHUNK_SIZE)
human_prots_chunks = [
    human_unique_prots_in_HVIDB[i * CHUNK_SIZE : (i + 1) * CHUNK_SIZE]
    for i in range(n_chunks)
]

human_prots_in_HVIDB_fasta_file = fasta.FastaFile()

for i, chunk in enumerate(human_prots_chunks):
    # # Introducing kind of a numerical progress bar
    # if i % 20 == 0:
    #     print(i)
    time.sleep(2)

    uniprot_query_unsuccessful = True
    while uniprot_query_unsuccessful:
        try:
            uniprot_entries_list = uniprot.fetch(
                ids=chunk,
                format="fasta"
            )

            for io_object in uniprot_entries_list:
                try:
                    current_fasta_file = fasta.FastaFile.read(io_object)
                    header, seq_str = list(current_fasta_file.items())[0]
                    human_prots_in_HVIDB_fasta_file[header] = seq_str
                except:
                    continue
            uniprot_query_unsuccessful = False
        except:
            time.sleep(1)

human_prots_in_HVIDB_fasta_file.write(
    "human_prots_in_HVIDB_VACV_WR_data_set.fasta"
)

In [4]:
# Investigate whether a UniProt entry has been retrieved for each and
# every ID
if len(human_prots_in_HVIDB_fasta_file) != len(human_unique_prots_in_HVIDB):
    human_prots_headers = human_prots_in_HVIDB_fasta_file.keys()
    uniprot_ids_in_fasta_file = [
        header.split("|")[1] for header in human_prots_headers
    ]

    n_prots_not_included = (
        len(human_unique_prots_in_HVIDB)
        -
        len(human_prots_in_HVIDB_fasta_file)
    )
    prots_not_included = []

    for uniprot_id in human_unique_prots_in_HVIDB:
        if uniprot_id not in uniprot_ids_in_fasta_file:
            prots_not_included.append(uniprot_id)
    
    print(
        f"For the following {n_prots_not_included} UniProt IDs, no "
        f"UniProt entry has been retrieved: {prots_not_included}"
    )

For the following 3 UniProt IDs, no UniProt entry has been retrieved: ['A0A087X117', 'A0A0A0MR88', 'A0A0A0MTS7', 'A0A3B3IS20', 'H0Y4G9']


In [17]:
# Surprisingly, although the FASTA file has only three proteins less
# than there are unique human proteins in the HVIDB data set, five
# UniProt IDs are found not to occur in the FASTA file
# This is subjected to further scrutiny
#
# A0A087X117: has indeed been removed from UniProt
#
# A0A0A0MR88: has indeed been removed from UniProt
#
# A0A0A0MTS7: the entry still exists, but has been merged into C9JQJ2
#
# A0A3B3IS20: the entry still exists, but has been merged into
# A0A3B3IRW5
#
# H0Y4G9: has indeed been removed from UniProt
#
# The DataFrame harbouring the HVIDB PPIs is updated to reflect these
# changes, following which the updated DataFrame is saved to disk
len_prior_to_filtering = len(HVIDB_VACV_WR_interactions_df)
print(
    "Length of HVIDB VACV WR-human PPI data set prior to filtering out "
    f"IDs removed from UniProt: {len_prior_to_filtering}"
)

HVIDB_VACV_WR_interactions_df = HVIDB_VACV_WR_interactions_df[
    ~(
        HVIDB_VACV_WR_interactions_df["Human-virus PPI"].str.contains(
            "A0A087X117"
        )
        |
        HVIDB_VACV_WR_interactions_df["Human-virus PPI"].str.contains(
            "A0A0A0MR88"
        )
        |
        HVIDB_VACV_WR_interactions_df["Human-virus PPI"].str.contains(
            "H0Y4G9"
        )
    )
]

assert len(HVIDB_VACV_WR_interactions_df) == (
    len_prior_to_filtering - 3
), (
    "Something went wrong while filtering out interactions involving "
    "proteins removed from UniProt!"
)

print(
    "Length of HVIDB VACV WR-human PPI data set after filtering out "
    f"IDs removed from UniProt: {len(HVIDB_VACV_WR_interactions_df)}"
)

Length of HVIDB VACV WR-human PPI data set prior to filtering out IDs removed from UniProt: 415
Length of HVIDB VACV WR-human PPI data set after filtering out IDs removed from UniProt: 412


In [21]:
# Now, update the IDs that have been merged into other ones
# To this end, iterate over the DataFrame rows
for i, row in HVIDB_VACV_WR_interactions_df.iterrows():
    row_human_prot_id = row["Human-virus PPI"].split("-")[0]
    row_virus_prot_id = row["Human-virus PPI"].split("-")[1]
    if row_human_prot_id == "A0A0A0MTS7":
        new_interaction_pair = "C9JQJ2" + "-" + row_virus_prot_id
        # Bear in mind that `.iterrows()` returns the rows' index, which
        # is not necessarily identical to the respective integer
        # positions
        # Hence, employing `.at()` is preferred to using `.iat()`
        HVIDB_VACV_WR_interactions_df.at[
            i, "Human-virus PPI"
        ] = new_interaction_pair
    elif row_human_prot_id == "A0A3B3IS20":
        new_interaction_pair = "A0A3B3IRW5" + "-" + row_virus_prot_id
        HVIDB_VACV_WR_interactions_df.at[
            i, "Human-virus PPI"
        ] = new_interaction_pair

In [22]:
# Finally, save the updated DataFrame to Disk
HVIDB_VACV_WR_interactions_df.to_csv(
    "all_HVIDB_VACV_WR_interactions.csv",
    index=False
)

In [ ]:
# Perform a final sanity check
list_of_human_prots_in_HVIDB_fasta_file = np.unique([
    int_pair.split("-")[0]
    for int_pair in HVIDB_VACV_WR_interactions_df["Human-virus PPI"]
]).tolist()

ids_not_supposed_to_occur = [
    "A0A087X117",
    "A0A0A0MR88",
    "A0A0A0MTS7",
    "A0A3B3IS20",
    "H0Y4G9"
]

assert all([
    outcast_id not in list_of_human_prots_in_HVIDB_fasta_file
    for outcast_id in ids_not_supposed_to_occur
]), (
    "Something went wrong while removing invalid IDs and replacing IDs "
    "that were merged into others!"
)

In [28]:
# Now, complete information transmission is also checked for the human
# interaction partners
# To this end, the reference proteome of Homo sapiens is downloaded and
# loaded into a DataFrame
homo_sapiens_ref_proteome_fasta = fasta.FastaFile.read(
    "uniprotkb_Homo_sapiens_reference_proteome_06_11_2024.fasta"
)
homo_sapiens_ref_proteome_dict = fasta.get_sequences(
    homo_sapiens_ref_proteome_fasta
)

human_prots_in_HVIDB_fasta_file = fasta.FastaFile.read(
    "human_prots_in_HVIDB_VACV_WR_data_set.fasta"
)

correct_seqs_list = [False] * len(human_prots_in_HVIDB_fasta_file)

for i, (header, seq_str) in enumerate(
    human_prots_in_HVIDB_fasta_file.items()
):
    try:
        ref_seq = str(homo_sapiens_ref_proteome_dict[header])
    except KeyError:
        print("One header was incompletely transmitted!")
        continue

    if seq_str == ref_seq:
        correct_seqs_list[i] = True

assert all(correct_seqs_list), (
    "Complete information transmission did not occur for all proteins!"
)

/opt/homebrew/Caskroom/miniforge/base/envs/bioinformatics/lib/python3.10/site-packages/biotite/sequence/io/fasta/convert.py:253: UserWarning: ProteinSequence objects do not support selenocysteine (U), occurrences were substituted by cysteine (C)
  warnings.warn(


One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!
One header was incompletely transmitted!


AssertionError: Complete information transmission did not occur for all proteins!

In [31]:
# The discrepancy of 17 proteins is entirely attributable to KeyErrors,
# i.e. the fact that the respective headers are not contained in the
# dictionary of the reference proteome
# Determine the UniProt IDs of those 17 proteins via Boolean indexing
human_prots_in_HVIDB_headers = list(
    human_prots_in_HVIDB_fasta_file.keys()
)
human_prots_in_HVIDB_IDs = [
    header.split("|")[1] for header in human_prots_in_HVIDB_headers
]

discrepancy_ids = np.array(
    human_prots_in_HVIDB_IDs
)[~np.array(correct_seqs_list)]

print(discrepancy_ids)

['A0N0N7' 'A0N0Q3' 'A3KPC7' 'A4FTV9' 'A8ASI8' 'B2R4P9' 'B2R4R0' 'B2R4S9'
 'B2RDW1' 'B2ZZ89' 'B4DJ51' 'D9YZV4' 'D9ZGF2' 'E5KTA5' 'E9KL37' 'F4ZW62'
 'Q548T7']


In [34]:
# Querying the UniProt database with the 17 IDs causing the discrepancy
# revealed both that they exist and there ID has not been changed
# Thus, they are simply not included in the reference proteome of Homo
# sapiens
# Complete information transmission is nevertheless checked by verifying
# that the sequences have the expected length
discrepancy_seq_lengths = [
    397, 127, 128, 130, 195, 136, 103, 126, 156, 2364, 149, 284, 3177,
    313, 1012, 390, 677
]

discrepancy_headers = np.array(human_prots_in_HVIDB_headers)[
    ~np.array(correct_seqs_list)
]

human_prots_in_HVIDB_dict = fasta.get_sequences(
    human_prots_in_HVIDB_fasta_file
)

correct_lengths_list = []

for header, ref_len in zip(discrepancy_headers, discrepancy_seq_lengths):
    transmitted_seq = str(human_prots_in_HVIDB_dict[header])
    correct_lengths_list.append(
        len(transmitted_seq) == ref_len
    )

assert all(correct_lengths_list), (
    f"For {len(correct_lengths_list) - sum(correct_lengths_list)} of "
    "the 17 IDs causing the discrepancy, the respective sequence has "
    "not been correctly transmitted."
)

In [ ]:
# I stopped here

In [ ]:
"""Code I cannot remember the purpose of underneath..."""

In [16]:
VACV_proteome_fasta = ref_proteome_fasta.copy()

# Querying the UniProtKB database with a couple of the IDs not included
# in the reference genome reveals that some IDs are unreviewed, while
# others have a low annotation score and where therefore not
# incorporated
# The functionalities provided by Biotite are again used in order to
# retrieve the respective proteins' sequences from the UniProt database,
# add them to the sequences from the reference proteome and save the
# resulting concatenation to a new FASTA file
# As a list of IDs is passed to `uniprot.fetch()` and `target_path` is
# not specified, a list of StringIO objects is returned which can be
# iterated over
for io_object in uniprot.fetch(outcast_protein_IDs, "fasta"):
    # Read the StringIO object into a Fasta file
    fasta_file = fasta.FastaFile.read(io_object)
    # Each file contains only one entry; hence, the first and only
    # element is retrieved from the the iterator returned by the 
    # `items()` method
    header, seq_str = list(fasta_file.items())[0]
    
    VACV_proteome_fasta[header] = seq_str

# Strangely enough, creating a deep copy of a FASTA file resets its
# length to zero although it is not empty (applies to Biotite version
# 0.39.0); hence, instead of evaluating whether the new file's length
# equals the old file's length plus the amount of new entries, it is
# checked whether the new file's length equals the amount of newly added
# items
assert len(VACV_proteome_fasta) == 18, (
    "Something went wrong while adding the 18 entries to the reference "
    "proteome!"
)

In [ ]:
# Remember that the UniProtKB (KB stands for knowledgebase) is composed
# of two sections, which are UniProtKB/Swiss-Prot and UniProtKB/TrEMBL
# The former is characterised by the fact that it contains exclusively
# reviewed, i.e. manually curated entries
# The latter, however, are unreviewed
# In order to incorporate both sections of UniProtKB, "UniProtKB" is
# chosen as the destination database ("To database")
# As source database ("from database"),
# "Gene annotation databases/Gene ID" is chosen
# Obtain the unique gene IDs
csv_df = pd.read_csv(
    "Vaccinia_Report_updated_and_errors_fixed.csv",
    sep="\t",
    dtype=dtype_dict
)

# Using those settings, 

# Using the parameters "from: UniProt/Gene Name",
# "to: UniProtKB-Swiss-Prot" and "taxId: 9606", 18,185 of the 18,396
# human protein IDs were mapped to 18,645 results, whereas 213 ID were
# not mapped

In [5]:
VACV_interactions_df = pd.read_csv(
    "all_HVIDB_VACV_interactions.csv"
)

interaction_pairs = VACV_interactions_df["Human-virus PPI"].to_list()
# Bear in mind that in the interaction pairs, the first ID preceding the
# hyphen represents the human protein, whereas the second ID following
# the hyphen represents the virus protein
human_uniprot_ids = [pair.split("-")[0] for pair in interaction_pairs]
unique_human_uniprot_ids = np.unique(human_uniprot_ids)

for uniprot_id in unique_human_uniprot_ids:
    print(uniprot_id)

A0A075B749
A0A087X117
A0A0A0MR88
A0A0A0MTS7
A0A0D9SG04
A0A0J9YX62
A0A0U1RRM6
A0A1B0GTL5
A0A2R8Y5A3
A0A3B3IS20
A0A3B3ISQ4
A0N0N7
A0N0Q3
A3KPC7
A4FTV9
A6NFX8
A6NNZ2
A7MCY6
A8ASI8
A8MUS3
B2R4P9
B2R4R0
B2R4S9
B2RDW1
B2ZZ89
B4DJ51
B4DLJ1
B8ZZN6
D9YZV4
D9ZGF2
E5KTA5
E9KL37
E9PDI4
F4ZW62
F8VVA7
F8VXC8
F8VZQ9
F8WBV6
G3V5R9
G5E9I4
H0Y4G9
H3BSR6
J3QK89
K7EQ78
K7ERV3
O00151
O00206
O00327
O00339
O00391
O00459
O00499
O00515
O00571
O00585
O14578
O14654
O14733
O14920
O14933
O15020
O15021
O15037
O15111
O15131
O15444
O43187
O43390
O43521
O43815
O60506
O60814
O75113
O75152
O75190
O75376
O75398
O75477
O75665
O94763
O95400
O95793
P00451
P01375
P01579
P01903
P02511
P02775
P02776
P04083
P04183
P04439
P04792
P04908
P05023
P05141
P05166
P05412
P06733
P06748
P07355
P07437
P07910
P08238
P09104
P09493
P09651
P0C0S8
P0CG48
P0DP23
P11021
P12036
P12111
P12235
P12236
P12273
P12277
P13051
P13637
P13861
P13929
P14923
P15924
P18583
P19013
P19237
P19525
P19876
P20042
P20671
P20700
P22362
P22492
P22531
P22532
P22626
P230

In [14]:
# Regarding the mapping of UniProt IDs of human interaction partners to
# gene names, some IDs seem to have been mapped to multiple names
# This is subjected to further scrutiny
human_mapping_df = pd.read_csv(
    "HVIDB_human_interaction_partner_mapping_to_gene_names.tsv",
    sep="\t"
)

unique_uniprot_ids = human_mapping_df["From"].unique()

multiple_mappings_list = []
for uniprot_id in unique_uniprot_ids:
    if (human_mapping_df["From"] == uniprot_id).sum() > 1:
        multiple_mappings_list.append(uniprot_id)

In [16]:
for uniprot_id in multiple_mappings_list:
    print(uniprot_id)

P04908
P0C0S8
P62805
P62807
P68431
P84243
Q6FI13
Q71DI3
